In [19]:
# 汎用ライブラリのimport
import sys
import os
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import statsmodels.api as sm
import math
import tensorflow as tf
import collections
from datetime import datetime
import csv
from itertools import chain

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import scipy.stats

In [54]:
# keras用ライブラリ
from keras.datasets import mnist
from keras.models import Sequential, model_from_json
from keras.layers import Dense, Activation
from keras.optimizers import SGD
from keras.utils import np_utils

In [4]:
# 自作ライブラリのimport
if os.environ['BR_HOME']+"/boatrace" not in sys.path:
    sys.path.append(os.environ['BR_HOME']+"/boatrace")
#print(sys.path)

from setup.myUtil import dbHandler

In [7]:
# 舟券の配列を取得
funakenList=[]
with open(os.environ['BR_HOME']+'/boatrace/config/3t_list.dat') as f:
    reader=csv.reader(f)
    for row in reader:
        funakenList.append(row)
funakenID = [i for i in range(120)]
funakenDict=dict(zip(funakenList[0],funakenID))

In [8]:
# 分析期間の指定は一旦ここでまとめてみる。
simStartDate="20190101"
simEndDate="20190531"

In [9]:
dbh=dbHandler.getDBHandle()
#dbHandler.closeDBHandle(dbh)

In [10]:
# データを取得
with dbh.cursor() as cursor:
    sel_sql = "select * from raceabst_forml_rentai_v \
               where raceDate between '%s' and '%s' \
               order by raceId "\
               % (simStartDate,simEndDate)
    cursor.execute(sel_sql)
    loadList=cursor.fetchall()
print("simdata:",len(loadList))

simdata: 22904


In [11]:
df = pd.io.json.json_normalize(loadList)
df.head()

,funaken,l1Fcnt,l1boat2r,l1boat3r,l1motor2r,l1motor3r,l1oldavgstdev,l1oldavgsttime,l1oldrank1,l1oldrank2,...,l6oldrank4,l6oldrank5,l6oldrank6,l6rank,l6score,odds,raceDate,raceId,raceWaveHeight,raceWindSpeed
0,1-2-5,0,0.4074,0.5556,0.3061,0.4558,0.00115,0.1810,0.04,0.14,...,0.22,0.23,0.18,B1,1.464794,14.8,2019-01-01,20190101-06-01,2,3.0
1,3-4-1,0,0.4717,0.6038,0.2687,0.4104,-0.02393,0.1625,0.18,0.15,...,0.21,0.14,0.19,B1,0.749802,63.8,2019-01-01,20190101-06-02,2,3.0
2,6-4-1,0,0.3704,0.5556,0.4875,0.6375,0.01078,0.1824,0.10,0.12,...,0.12,0.16,0.16,A1,8.404421,29.9,2019-01-01,20190101-06-03,2,3.0
3,1-3-6,0,0.3704,0.5370,0.3916,0.5060,0.00080,0.1676,0.33,0.17,...,0.26,0.14,0.17,B1,1.101158,5.3,2019-01-01,20190101-06-04,2,3.0
4,6-2-3,0,0.4082,0.5102,0.2568,0.4865,0.02388,0.2003,0.00,0.04,...,0.12,0.10,0.09,A1,3.301202,16.3,2019-01-01,20190101-06-05,2,3.0


In [14]:
# 入力のデータ整形
xdf=df.drop(['funaken','odds','raceId','raceDate'],axis=1)
# オッズから作ったスコアは効きすぎるので捨ててみる
xdf=xdf.drop(['l1score','l2score','l3score','l4score','l5score','l6score'],axis=1)
#xdf=xdf.drop(['l1Fcnt','l2Fcnt','l3Fcnt','l4Fcnt','l5Fcnt','l6Fcnt'],axis=1)
#xdf=xdf.drop(['l1oldavgstdev','l2oldavgstdev','l3oldavgstdev','l4oldavgstdev','l5oldavgstdev','l6oldavgstdev'],axis=1)

rankLabel=LabelEncoder()
rankLabel=rankLabel.fit(xdf['l1rank'])
xdf['l1rank']=rankLabel.transform(xdf['l1rank'])
xdf['l2rank']=rankLabel.transform(xdf['l2rank'])
xdf['l3rank']=rankLabel.transform(xdf['l3rank'])
xdf['l4rank']=rankLabel.transform(xdf['l4rank'])
xdf['l5rank']=rankLabel.transform(xdf['l5rank'])
xdf['l6rank']=rankLabel.transform(xdf['l6rank'])
xdf.head()

,l1Fcnt,l1boat2r,l1boat3r,l1motor2r,l1motor3r,l1oldavgstdev,l1oldavgsttime,l1oldrank1,l1oldrank2,l1oldrank3,...,l6oldavgsttime,l6oldrank1,l6oldrank2,l6oldrank3,l6oldrank4,l6oldrank5,l6oldrank6,l6rank,raceWaveHeight,raceWindSpeed
0,0,0.4074,0.5556,0.3061,0.4558,0.00115,0.1810,0.04,0.14,0.31,...,0.1867,0.09,0.15,0.13,0.22,0.23,0.18,2,2,3.0
1,0,0.4717,0.6038,0.2687,0.4104,-0.02393,0.1625,0.18,0.15,0.13,...,0.1730,0.18,0.12,0.16,0.21,0.14,0.19,2,2,3.0
2,0,0.3704,0.5556,0.4875,0.6375,0.01078,0.1824,0.10,0.12,0.13,...,0.1254,0.22,0.18,0.16,0.12,0.16,0.16,0,2,3.0
3,0,0.3704,0.5370,0.3916,0.5060,0.00080,0.1676,0.33,0.17,0.20,...,0.2045,0.08,0.18,0.17,0.26,0.14,0.17,2,2,3.0
4,0,0.4082,0.5102,0.2568,0.4865,0.02388,0.2003,0.00,0.04,0.04,...,0.1540,0.31,0.17,0.21,0.12,0.10,0.09,0,2,3.0


In [21]:
# normalizeしておく
# ★validationの時に要注意。
xdf=xdf.apply(scipy.stats.zscore, axis=0)

In [22]:
xdf.head()

,l1Fcnt,l1boat2r,l1boat3r,l1motor2r,l1motor3r,l1oldavgstdev,l1oldavgsttime,l1oldrank1,l1oldrank2,l1oldrank3,...,l6oldavgsttime,l6oldrank1,l6oldrank2,l6oldrank3,l6oldrank4,l6oldrank5,l6oldrank6,l6rank,raceWaveHeight,raceWindSpeed
0,-0.404206,0.802886,0.539349,-0.160546,-0.194103,0.093684,-0.050510,-1.539874,-0.735871,2.929335,...,-0.018185,-0.587633,-0.042491,-0.400240,1.053744,0.993327,-0.078889,0.559703,-0.292066,0.039602
1,-0.404206,1.415346,0.899224,-0.476856,-0.497122,-0.506313,-0.430154,-0.117148,-0.553387,-0.928423,...,-0.299386,0.283559,-0.451575,0.068753,0.882839,-0.391612,-0.012798,0.559703,-0.292066,0.039602
2,-0.404206,0.450459,0.539349,1.373641,1.018642,0.324065,-0.021780,-0.930134,-1.100839,-0.928423,...,-1.276403,0.670756,0.366593,0.068753,-0.655300,-0.083848,-0.211073,-1.491899,-0.292066,0.039602
3,-0.404206,0.450459,0.400476,0.562569,0.140954,0.085310,-0.325495,1.407201,-0.188419,0.571816,...,0.347170,-0.684432,0.366593,0.225084,1.737362,-0.391612,-0.144981,0.559703,-0.292066,0.039602
4,-0.404206,0.810506,0.200380,-0.577500,0.010802,0.637461,0.345551,-1.946367,-2.560709,-2.857302,...,-0.689372,1.541948,0.230232,0.850409,-0.655300,-1.007140,-0.673715,-1.491899,-0.292066,0.039602


In [35]:
# ファイルから作った辞書で変換する
ydf=df['funaken']
ydf=pd.DataFrame(ydf.replace(funakenDict))
ydf['funaken']=ydf['funaken'].astype(int)
#ydf.head()

# ydfはone-hotにしておく。
y=np.eye(120)[ydf['funaken']]
print(y)

[[0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [100]:
# 重み付けのため、オッズのリストを作る
odf=df['odds'].values
raceId_df=df['raceId'].values
#odf=pd.DataFrame(df['odds'])
#odf.describe()
print(odf)
print(raceId_df)

[14.8 63.8 29.9 ...  6.  15.7  5.3]
['20190101-06-01' '20190101-06-02' '20190101-06-03' ... '20190531-23-10'
 '20190531-23-11' '20190531-23-12']


In [126]:
#train/testを分割
X_train, X_test, y_train, y_test,o_train,o_test,raceId_train,raceId_test = train_test_split(xdf, y,odf,raceId_df)
print("X_train,X_test:",len(X_train),len(X_test))
print(type(X_train))
print("y_train,y_test:",len(y_train),len(y_test))
print(type(y_train))
print("o_train,o_test:",len(o_train),len(o_test))
print(type(o_train))
print("raceId_train,raceId_test:",len(raceId_train),len(raceId_test))
print(type(o_train))

X_train,X_test: 17178 5726
<class 'pandas.core.frame.DataFrame'>
y_train,y_test: 17178 5726
<class 'numpy.ndarray'>
o_train,o_test: 17178 5726
<class 'numpy.ndarray'>
raceId_train,raceId_test: 17178 5726
<class 'numpy.ndarray'>


In [128]:
# class
class my_Net:
    def __init__(self):
        self.Epoch = 200
        self.Batch_size = 8000
        self.Verbose = 1 #ログの出力モード切替 : 1 プログレスバーで表示
        self.output_size = 120
        self.optimize = SGD()
        #self.hidden_units = 128
        self.hidden_units = 256
        self.Validation_split = 0.2 #訓練データの中で検証データとして扱う割合
        #self.Reshape = 28 * 28
        
        self.model = Sequential()
 
    def make_net(self):
        self.model.add(Dense(self.hidden_units, input_shape=(86,)))
        self.model.add(Activation("relu"))
        self.model.add(Dense(self.hidden_units))
        self.model.add(Activation("relu"))
        self.model.add(Dense(self.hidden_units))
        self.model.add(Activation("relu"))
        self.model.add(Dense(self.hidden_units))
        self.model.add(Activation("relu"))
        self.model.add(Dense(self.output_size))
        self.model.add(Activation("softmax"))
        self.model.summary()
 
    def model_compile(self):
        self.model.compile(loss="categorical_crossentropy", #交差エントロピー誤差関数
                           optimizer=self.optimize,#確率的勾配下降法
                           metrics=["accuracy"])
 
    def make_model(self):
        self.make_net()
        self.model_compile()
 
    def train(self, x, t):
        self.model.fit(x, t, batch_size=self.Batch_size,
                       epochs=self.Epoch,
                       verbose=self.Verbose,
                       validation_split=self.Validation_split)
 
    def score(self, x, t):
        return self.model.evaluate(x, t, verbose=self.Verbose)



In [129]:
net = my_Net()
 
net.make_model()
 
net.train(X_train, y_train)
    
score = net.score(X_test, y_test)
 
print("\nTest loss:", score[0])
print("\nTest accuracy:", score[1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_78 (Dense)             (None, 256)               22272     
_________________________________________________________________
activation_78 (Activation)   (None, 256)               0         
_________________________________________________________________
dense_79 (Dense)             (None, 256)               65792     
_________________________________________________________________
activation_79 (Activation)   (None, 256)               0         
_________________________________________________________________
dense_80 (Dense)             (None, 256)               65792     
_________________________________________________________________
activation_80 (Activation)   (None, 256)               0         
_________________________________________________________________
dense_81 (Dense)             (None, 256)               65792     
__________

Epoch 48/200
13742/13742 [==============================] - 0s 28us/step - loss: 4.4678 - acc: 0.0725 - val_loss: 4.4730 - val_acc: 0.0701
Epoch 49/200
13742/13742 [==============================] - 0s 28us/step - loss: 4.4618 - acc: 0.0725 - val_loss: 4.4674 - val_acc: 0.0701
Epoch 50/200
13742/13742 [==============================] - 0s 28us/step - loss: 4.4559 - acc: 0.0725 - val_loss: 4.4618 - val_acc: 0.0701
Epoch 51/200
13742/13742 [==============================] - 0s 28us/step - loss: 4.4500 - acc: 0.0725 - val_loss: 4.4563 - val_acc: 0.0701
Epoch 52/200
13742/13742 [==============================] - 0s 28us/step - loss: 4.4443 - acc: 0.0725 - val_loss: 4.4510 - val_acc: 0.0701
Epoch 53/200
13742/13742 [==============================] - 0s 28us/step - loss: 4.4387 - acc: 0.0725 - val_loss: 4.4458 - val_acc: 0.0701
Epoch 54/200
13742/13742 [==============================] - 0s 28us/step - loss: 4.4332 - acc: 0.0725 - val_loss: 4.4407 - val_acc: 0.0701
Epoch 55/200
13742/13742 [=

Epoch 107/200
13742/13742 [==============================] - 0s 28us/step - loss: 4.2686 - acc: 0.0718 - val_loss: 4.2913 - val_acc: 0.0693
Epoch 108/200
13742/13742 [==============================] - 0s 28us/step - loss: 4.2670 - acc: 0.0719 - val_loss: 4.2899 - val_acc: 0.0693
Epoch 109/200
13742/13742 [==============================] - 0s 28us/step - loss: 4.2655 - acc: 0.0718 - val_loss: 4.2886 - val_acc: 0.0693
Epoch 110/200
13742/13742 [==============================] - 0s 28us/step - loss: 4.2640 - acc: 0.0718 - val_loss: 4.2872 - val_acc: 0.0687
Epoch 111/200
13742/13742 [==============================] - 0s 28us/step - loss: 4.2625 - acc: 0.0718 - val_loss: 4.2859 - val_acc: 0.0690
Epoch 112/200
13742/13742 [==============================] - 0s 28us/step - loss: 4.2610 - acc: 0.0718 - val_loss: 4.2846 - val_acc: 0.0690
Epoch 113/200
13742/13742 [==============================] - 0s 28us/step - loss: 4.2596 - acc: 0.0718 - val_loss: 4.2834 - val_acc: 0.0690
Epoch 114/200
13742/

13742/13742 [==============================] - 0s 28us/step - loss: 4.2096 - acc: 0.0726 - val_loss: 4.2397 - val_acc: 0.0681
Epoch 166/200
13742/13742 [==============================] - 0s 28us/step - loss: 4.2089 - acc: 0.0727 - val_loss: 4.2391 - val_acc: 0.0681
Epoch 167/200
13742/13742 [==============================] - 0s 28us/step - loss: 4.2083 - acc: 0.0725 - val_loss: 4.2385 - val_acc: 0.0681
Epoch 168/200
13742/13742 [==============================] - 0s 28us/step - loss: 4.2076 - acc: 0.0726 - val_loss: 4.2379 - val_acc: 0.0684
Epoch 169/200
13742/13742 [==============================] - 0s 28us/step - loss: 4.2069 - acc: 0.0728 - val_loss: 4.2374 - val_acc: 0.0684
Epoch 170/200
13742/13742 [==============================] - 0s 28us/step - loss: 4.2063 - acc: 0.0730 - val_loss: 4.2368 - val_acc: 0.0684
Epoch 171/200
13742/13742 [==============================] - 0s 28us/step - loss: 4.2056 - acc: 0.0727 - val_loss: 4.2363 - val_acc: 0.0684
Epoch 172/200
13742/13742 [=======

In [57]:
#モデルを保存
model_path=os.path.join(os.environ['BR_HOME'],"boatrace/models")
model_modelfile=os.path.join(model_path,'keras_perceptron_model.json')
model_weightfile=os.path.join(model_path,'keras_perceptron_weight.json')

open(model_modelfile,"w").write(net.model.to_json())
net.model.save_weights(model_weightfile)

In [130]:
# testの回収率をシミュレート
# オッズを見て判断する場合
resAmount=0
buyAmount=0
resCnt=0
buyCnt=0

testPredictList=net.model.predict(X_test,batch_size=len(X_test))

for i in range(len(raceId_test) ):
    raceId=raceId_train[i]
    with dbh.cursor() as cursor:
        sel_sql = "select funaken,odds from raceodds \
                   where oddsType = '3t' \
                   and raceId = '%s' \
                   order by funaken" \
                   % (raceId)

        cursor.execute(sel_sql)
        loadList=pd.DataFrame(cursor.fetchall())
        loadList=pd.DataFrame(loadList.replace(funakenDict))
                
    for j in range(120):
        # y_predの閾値を下げてみる。
        if testPredictList[i][j]> 0.08 and ( (testPredictList[i][j] * (loadList[loadList['funaken']==j]['odds'])).values[0] > 1.5) :
            print("buy:",raceId,i,j,loadList[loadList['funaken']==j]['odds'].values[0],round(testPredictList[i][j],3) )
            buyAmount+=1
            buyCnt+=1
            if list(y_test[i]).index(1)==j:
                print("☆hit!☆:",raceId,j,loadList[loadList['funaken']==j]['odds'].values[0])
                resAmount += o_test[i]
                resCnt+=1
            else:
                pass
#res=net.model.predict_on_batch(xdf[1:3])


buy: 20190220-13-07 4 0 49.6 0.092
buy: 20190220-13-07 4 5 25.4 0.08
buy: 20190514-10-09 7 0 94.9 0.107
buy: 20190317-14-01 8 0 17.8 0.12
buy: 20190317-11-08 10 0 23.8 0.084
buy: 20190425-06-03 13 0 21.4 0.084
buy: 20190509-15-03 21 0 24.3 0.086
buy: 20190103-22-06 31 0 45.5 0.091
buy: 20190418-01-02 35 0 56.6 0.101
buy: 20190418-01-02 35 1 35.4 0.089
buy: 20190313-03-11 36 1 29.4 0.084
buy: 20190527-06-05 45 0 19.0 0.206
buy: 20190415-09-09 51 0 31.5 0.097
☆hit!☆: 20190415-09-09 0 31.5
buy: 20190415-09-09 51 1 23.0 0.092
buy: 20190419-19-02 52 0 23.9 0.106
buy: 20190419-19-02 52 1 40.9 0.091
buy: 20190417-12-12 60 0 43.3 0.149
buy: 20190417-12-12 60 1 33.7 0.147
☆hit!☆: 20190417-12-12 1 33.7
buy: 20190404-15-06 61 0 31.4 0.103
buy: 20190502-01-10 66 0 22.0 0.109
buy: 20190119-07-03 75 0 22.0 0.092
buy: 20190203-11-09 76 0 43.1 0.093
buy: 20190105-05-01 78 0 22.7 0.081
buy: 20190423-11-11 79 0 16.0 0.119
buy: 20190106-03-11 80 0 11.9 0.127
buy: 20190412-09-03 86 0 22.0 0.081
buy: 20190

buy: 20190414-20-06 920 0 25.9 0.091
buy: 20190112-20-03 924 0 17.6 0.12
buy: 20190214-14-03 930 0 15.1 0.101
buy: 20190421-24-04 935 1 19.9 0.084
buy: 20190421-24-04 935 5 67.2 0.086
buy: 20190402-15-12 941 0 22.6 0.081
buy: 20190504-19-06 942 0 96.5 0.095
buy: 20190309-15-06 949 0 41.0 0.099
buy: 20190430-13-07 951 1 45.4 0.091
buy: 20190211-14-02 965 0 42.8 0.092
buy: 20190427-07-03 966 0 15.3 0.118
buy: 20190104-07-08 970 1 43.3 0.103
buy: 20190326-22-05 973 1 28.6 0.099
buy: 20190516-16-01 976 0 19.3 0.089
buy: 20190513-22-09 978 0 33.0 0.094
buy: 20190405-07-07 981 0 29.9 0.081
buy: 20190212-21-10 983 0 46.5 0.14
buy: 20190212-21-10 983 1 25.0 0.086
buy: 20190329-22-03 992 0 16.7 0.102
buy: 20190329-22-03 992 1 46.4 0.089
buy: 20190505-10-07 1010 0 26.1 0.121
buy: 20190505-10-07 1010 1 36.8 0.086
☆hit!☆: 20190505-10-07 1 36.8
buy: 20190311-03-01 1018 0 15.0 0.116
buy: 20190311-03-01 1018 1 35.6 0.099
buy: 20190302-06-03 1019 0 34.0 0.113
buy: 20190318-16-07 1022 0 64.3 0.132
buy:

buy: 20190509-20-02 1813 0 24.7 0.135
buy: 20190331-17-02 1814 0 19.3 0.121
buy: 20190408-09-09 1816 1 16.0 0.109
buy: 20190323-12-01 1826 0 22.7 0.086
buy: 20190316-05-04 1829 0 32.2 0.082
buy: 20190524-07-09 1836 0 14.7 0.11
☆hit!☆: 20190524-07-09 0 14.7
buy: 20190531-01-01 1837 0 16.5 0.095
buy: 20190531-01-01 1837 1 19.2 0.081
buy: 20190130-15-06 1841 0 26.6 0.093
buy: 20190130-15-06 1841 4 30.4 0.081
buy: 20190325-15-03 1857 0 50.1 0.136
buy: 20190205-11-09 1865 0 39.1 0.092
buy: 20190510-18-08 1867 0 18.6 0.084
☆hit!☆: 20190510-18-08 0 18.6
buy: 20190501-08-04 1870 0 37.1 0.089
buy: 20190415-12-09 1881 0 13.0 0.136
buy: 20190112-13-07 1890 0 54.7 0.173
buy: 20190112-13-07 1890 1 29.4 0.118
buy: 20190113-08-06 1891 0 36.0 0.115
buy: 20190113-08-06 1891 1 54.8 0.086
buy: 20190118-13-04 1892 0 31.4 0.14
buy: 20190211-07-04 1898 0 108.8 0.109
buy: 20190529-04-07 1904 0 30.9 0.184
buy: 20190529-04-07 1904 1 82.2 0.083
buy: 20190121-19-03 1921 0 28.9 0.135
buy: 20190121-19-03 1921 1 35

buy: 20190330-04-07 2728 0 58.8 0.104
buy: 20190413-20-04 2731 1 34.0 0.094
buy: 20190421-10-07 2733 0 24.3 0.115
buy: 20190105-05-07 2734 0 53.7 0.082
buy: 20190106-19-07 2735 0 20.4 0.09
buy: 20190505-19-04 2736 0 12.4 0.123
☆hit!☆: 20190505-19-04 0 12.4
buy: 20190302-07-11 2739 0 18.6 0.086
buy: 20190504-21-02 2752 0 89.2 0.137
☆hit!☆: 20190504-21-02 0 89.2
buy: 20190504-21-02 2752 1 38.1 0.11
buy: 20190315-24-01 2753 0 18.9 0.119
buy: 20190206-15-03 2763 0 78.6 0.086
buy: 20190417-23-09 2768 0 22.4 0.094
buy: 20190320-15-04 2779 0 33.7 0.164
buy: 20190320-15-04 2779 1 91.8 0.084
buy: 20190414-12-06 2780 0 16.9 0.129
buy: 20190320-01-11 2781 1 21.9 0.091
buy: 20190422-02-09 2787 0 16.4 0.114
buy: 20190422-02-09 2787 1 35.8 0.099
buy: 20190221-03-08 2790 0 35.4 0.098
☆hit!☆: 20190221-03-08 0 35.4
buy: 20190117-15-02 2793 0 34.4 0.102
buy: 20190326-14-06 2795 0 29.2 0.187
☆hit!☆: 20190326-14-06 0 29.2
buy: 20190224-13-03 2805 0 28.9 0.081
buy: 20190125-15-03 2811 0 50.7 0.101
buy: 201

buy: 20190512-01-02 3658 0 39.7 0.083
buy: 20190407-23-10 3669 1 29.3 0.088
buy: 20190123-04-10 3681 0 14.3 0.115
buy: 20190130-01-02 3683 0 21.2 0.081
buy: 20190418-17-02 3687 0 25.1 0.092
buy: 20190313-03-02 3692 0 20.5 0.088
☆hit!☆: 20190313-03-02 0 20.5
buy: 20190320-01-06 3693 0 17.4 0.122
buy: 20190207-15-03 3695 0 181.0 0.092
buy: 20190422-07-03 3709 0 72.3 0.095
buy: 20190422-07-03 3709 1 98.6 0.082
buy: 20190116-11-07 3717 0 97.5 0.106
buy: 20190412-07-05 3722 0 56.5 0.122
buy: 20190504-04-02 3738 0 54.3 0.084
buy: 20190117-21-05 3741 0 26.3 0.126
buy: 20190318-18-06 3751 0 18.3 0.085
buy: 20190122-23-08 3752 1 23.3 0.082
buy: 20190118-01-01 3759 0 21.5 0.08
buy: 20190319-01-08 3772 0 23.1 0.124
buy: 20190409-23-08 3778 0 28.9 0.093
buy: 20190409-23-08 3778 4 39.3 0.082
buy: 20190110-23-05 3779 0 38.1 0.09
☆hit!☆: 20190110-23-05 0 38.1
buy: 20190202-10-06 3781 0 27.3 0.091
buy: 20190305-12-02 3782 0 35.6 0.092
buy: 20190305-12-02 3782 1 44.3 0.08
buy: 20190407-05-07 3787 0 18.

buy: 20190112-05-01 4538 0 16.6 0.112
buy: 20190512-22-10 4544 0 12.1 0.13
buy: 20190512-22-10 4544 1 18.8 0.101
buy: 20190112-02-11 4545 0 28.2 0.11
buy: 20190304-14-12 4548 0 8.8 0.214
buy: 20190103-02-05 4550 0 143.8 0.098
buy: 20190405-19-11 4556 0 26.1 0.122
buy: 20190405-19-11 4556 1 15.2 0.1
buy: 20190522-22-02 4563 0 22.2 0.107
buy: 20190224-21-07 4566 0 33.8 0.092
buy: 20190110-14-12 4569 1 16.3 0.102
buy: 20190407-05-10 4572 0 33.7 0.105
buy: 20190419-16-07 4574 0 22.6 0.095
buy: 20190515-22-01 4578 0 18.3 0.142
buy: 20190515-22-01 4578 1 15.9 0.099
buy: 20190529-19-04 4583 0 32.2 0.09
buy: 20190529-19-04 4583 4 63.0 0.082
buy: 20190103-08-04 4585 0 41.1 0.106
buy: 20190509-15-07 4586 0 64.3 0.123
buy: 20190509-15-07 4586 1 53.2 0.08
buy: 20190507-24-02 4594 0 18.3 0.083
buy: 20190210-06-05 4597 0 21.4 0.081
buy: 20190403-13-06 4605 0 107.4 0.099
buy: 20190108-05-04 4622 0 39.0 0.109
buy: 20190108-05-04 4622 5 41.0 0.081
buy: 20190406-02-07 4624 0 16.7 0.093
buy: 20190125-11-

buy: 20190123-17-06 5334 0 17.2 0.152
buy: 20190428-14-10 5346 0 27.2 0.121
buy: 20190427-18-05 5352 0 23.5 0.087
buy: 20190109-14-12 5353 1 17.6 0.095
buy: 20190511-01-02 5362 0 84.7 0.115
buy: 20190511-01-02 5362 1 192.1 0.102
buy: 20190523-22-04 5364 0 19.9 0.09
buy: 20190322-03-05 5365 0 32.5 0.081
buy: 20190121-04-06 5369 0 145.4 0.087
buy: 20190412-14-04 5376 0 42.1 0.102
buy: 20190307-24-07 5377 0 37.4 0.153
buy: 20190307-24-07 5377 1 31.9 0.109
buy: 20190521-17-09 5380 0 24.3 0.082
buy: 20190330-18-07 5381 0 25.6 0.109
buy: 20190315-04-02 5383 0 102.6 0.134
buy: 20190315-04-02 5383 1 159.4 0.082
buy: 20190118-03-05 5384 0 39.0 0.138
buy: 20190118-03-05 5384 1 49.7 0.081
buy: 20190425-02-04 5385 0 31.2 0.144
buy: 20190204-03-06 5388 1 22.8 0.166
buy: 20190204-03-06 5388 4 21.7 0.092
buy: 20190204-03-06 5388 5 56.0 0.084
buy: 20190102-03-11 5390 0 16.0 0.183
buy: 20190416-19-08 5394 0 15.3 0.108
buy: 20190416-19-08 5394 1 65.3 0.084
buy: 20190330-14-02 5397 0 20.4 0.119
buy: 2019

In [131]:
print("resultReturn:",resAmount/buyAmount)
print("totalRace,buy,return",len(y_test),buyAmount,resAmount )

resultReturn: 0.7087349397590362
totalRace,buy,return 5726 1328 941.2000000000002
